In [1]:
from enum import Enum

class FoodType(Enum):
    CANNED = 0
    VEG_OR_FRUIT = 1
    MEAT = 2


class StorageType(Enum):
    REFRIGERATE = 0
    FROZEN = 1
    NORMAL_TEMP = 2

class DaysInFridge(Enum):
    SHORT = 0
    MEDIUM = 1
    LONG = 2


# --- Food class ---
class Food:
    def __init__(self, name: str, food_type: FoodType, days_in_fridge: int, storage_type: StorageType):
        self.name = name
        self.food_type = food_type
        self.days_in_fridge = days_in_fridge
        self.storage_type = storage_type

    def __str__(self):
        return (f"{self.name} ({self.food_type.value}) stored as {self.storage_type.value} "
                f"since {self.date_in_fridge}")
    

In [2]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD

e:\School-Work-5-Y3S1\CS3263\Group66-Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
SampleFoodBN = DiscreteBayesianNetwork(
    [("Food Type", "Expired"), ("Storage Type", "Expired"), ("Days In Fridge", "Expired")]
)

In [4]:
cpd_FT = TabularCPD(
    variable="Food Type", variable_card=3, values=[[1 / 3], [1 / 3], [1 / 3]]
)

cpd_ST = TabularCPD(
    variable="Storage Type", variable_card=3, values=[[1 / 3], [1 / 3], [1 / 3]]
)

cpd_DF= TabularCPD(
    variable="Days In Fridge", variable_card=3, values=[[1 / 3], [1 / 3], [1 / 3]]
)

In [5]:
cpd_expired = TabularCPD(
    variable='Expired',
    variable_card=3,
    values=[
    # P(Fresh)
    [0.497, 0.490, 0.479, 0.499, 0.497, 0.494, 0.493, 0.478, 0.455,
     0.448, 0.342, 0.274, 0.485, 0.455, 0.410, 0.388, 0.315, 0.225,
     0.337, 0.275, 0.150, 0.548, 0.350, 0.112, 0.315, 0.075, 0.015],  

    # P(Near_expiry)
    [0.500, 0.500, 0.500, 0.500, 0.500, 0.500, 0.500, 0.500, 0.500,
     0.400, 0.350, 0.350, 0.500, 0.450, 0.420, 0.400, 0.375, 0.350,
     0.450, 0.300, 0.250, 0.450, 0.450, 0.350, 0.450, 0.445, 0.060,],  

    # P(Expired)
    [0.003, 0.010, 0.021, 0.001, 0.003, 0.006, 0.007, 0.022, 0.045,
     0.152, 0.308, 0.376, 0.015, 0.095, 0.170, 0.212, 0.310, 0.425,
     0.213, 0.425, 0.600, 0.002, 0.200, 0.538, 0.235, 0.480, 0.925]  
],
    evidence=["Food Type", "Storage Type", "Days In Fridge"],
    evidence_card=[3, 3, 3]
)

In [7]:
import pandas as pd
from pgmpy.factors.discrete import TabularCPD

# 1. Load the dataset
df = pd.read_csv("../food_condition.csv")

# 2. Define mapping to ensure consistent ordering with enums
food_order = ["CANNED", "VEG_OR_FRUIT", "MEAT"]
storage_order = ["REFRIGERATE", "FROZEN", "NORMAL_TEMP"]
days_order = ["SHORT", "MEDIUM", "LONG"]
condition_order = ["Fresh", "Near-expired", "Expired"]

# 3. Compute conditional probabilities
probs = []
for food in food_order:
    for storage in storage_order:
        for days in days_order:
            subset = df[
                (df["FoodType"] == food)
                & (df["StorageType"] == storage)
                & (df["DaysInFridge"] == days)
            ]
            # Sum counts per condition
            counts = subset.groupby("Condition")["Count"].sum().reindex(condition_order, fill_value=0)
            total = counts.sum()
            probs.append((counts / total).tolist())

# 4. Reorganize into rows = conditions, columns = combinations of evidence
values = list(map(list, zip(*probs)))  # transpose to shape (3, 27)

# 5. Create the CPD
cpd_expired = TabularCPD(
    variable="Expired",
    variable_card=3,
    values=values,
    evidence=["Food Type", "Storage Type", "Days In Fridge"],
    evidence_card=[3, 3, 3]
)

# 6. Print CPD for inspection
print(cpd_expired)

+----------------+-----+-------------------+
| Food Type      | ... | Food Type(2)      |
+----------------+-----+-------------------+
| Storage Type   | ... | Storage Type(2)   |
+----------------+-----+-------------------+
| Days In Fridge | ... | Days In Fridge(2) |
+----------------+-----+-------------------+
| Expired(0)     | ... | 0.125             |
+----------------+-----+-------------------+
| Expired(1)     | ... | 0.3125            |
+----------------+-----+-------------------+
| Expired(2)     | ... | 0.5625            |
+----------------+-----+-------------------+


In [8]:
SampleFoodBN.add_cpds(cpd_FT, cpd_ST, cpd_DF, cpd_expired)

In [9]:
SampleFoodBN.check_model()

True

In [10]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(SampleFoodBN)

In [11]:
p = infer.query(["Expired"], evidence={"Food Type": 1, "Storage Type": 0, "Days In Fridge": 2})
print(p)

+------------+----------------+
| Expired    |   phi(Expired) |
+============+================+
| Expired(0) |         0.4375 |
+------------+----------------+
| Expired(1) |         0.3125 |
+------------+----------------+
| Expired(2) |         0.2500 |
+------------+----------------+


In [12]:
print(p.values)

[0.4375 0.3125 0.25  ]
